# Local Search

For some problems, the path from the initial state to the solution is not that important. What matters is the goal state! For such problems, we may be able to do better that the systematic state-space exploration of the *classical* search algorithms. Local search algorithms try to take advantage of that and that enables them to achieve VERY low (sometimes constant) memory requirements for even the biggest problems!

## Problem Formulation

Problem formulation, again, is the first step in solving it! We will be using heuristic functions like informed search!

In [ ]:
class Problem:
    '''
    Abstract base class for problem formulation that supports a heuristic function.
    It declares the expected methods to be used by a search algorithm.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''

    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state and, if applicable, the goal state.'''
        self.init_state = None

    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError

    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError

    def goal_test(self, state):
        '''Returns whether or not the given state is a goal state.'''
        raise NotImplementedError

    def step_cost(self, state, action):
        '''Returns the step cost of applying the given action to the given state.'''
        raise NotImplementedError

    def heuristic(self, state):
        '''Returns the heuristic value of the given state, i.e., the estimated number of steps to the nearest goal state.'''
        raise NotImplementedError

    @classmethod
    def new_random_instance(cls):
        '''Factory method to a problem instance with a random initial state.'''
        raise NotImplementedError


**SELF-CHECK:** Do we need the *Node* structure in local search algorithms?

## Visualization

As always, let's write some code to help us visualize the problem! It is early this time because we are going to use it in multiple algorithms!

In [ ]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for state in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, state)
        print('-' * terminal_width)

## Example Problem: 8-Queens Puzzle

We will be using a rather famous problem, the [8-queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle), where 8 queens should be placed on a chess board with no queen attacking another (no more than one queen in each row, column or diagonal). This is a classical problem where the path to the solution is irrelevant; the final queens configuration is all that matters!

In [ ]:
from random import randint

class EightQueens(Problem):
    '''8-Queens Puzzle problem formulation.'''

    def __init__(self, init_state):
        # The state is represented as a 8-element tuple of integers where each number represents the placement of a queen in a column
        # This representation automatically imposes that there are exactly one queen in each column
        self.init_state = init_state

    def actions(self, state):
        actions = []
        for i, placement in enumerate(state):  # Iterating through all columns
            for new_placement in range(1, 9):  # Iterating through all possible placements in a column
                if new_placement is not placement:  # Excluding the current placement
                    actions.append((i, new_placement))  # An action is represented as changing the queen in column i to a new placement
        return actions

    def result(self, state, action):
        new_state = list(state)  # Creating a new MUTABLE list from the current state
        new_state[action[0]] = action[1]  # Applying the action by changing the placement in a column
        return tuple(new_state)  # Casting back to the IMMUTABLE tuple state representation

    def goal_test(self, state):
        return self.heuristic(state) is 0  # The goal is reached (in this specific problem) when the heuristic function is 0

    def step_cost(self, state, action):
        return 1  # All moves have unit cost

    def heuristic(self, state):
        attacking_queen_pairs = 0  # Number of pairs of queens attacking each other
        for i, placement1 in enumerate(state):  # Iterating through all columns
            for j, placement2 in enumerate(state[i + 1:]):  # Iterating through all columns after column i
                if placement1 is placement2 or abs(int(placement1) - int(placement2)) is j + 1:  #  Adding an attacking queen pair if both are in the same column or diagonal
                    attacking_queen_pairs += 1
        return attacking_queen_pairs

    @classmethod
    def new_random_instance(cls):
        return cls(tuple(randint(1, 8) for _ in range(8)))  # A tuple of 8 random numbers representing 8 queens randomly


def _eight_queens_visualizer(problem, state):
    '''Custom visualizer for the eight queens puzzle problem.'''
    for i in range(1, 9):
        for j in range(8):
            print('⬛' if state[j] is i else '⬜', end='')
        print()

_visualizers[EightQueens] = _eight_queens_visualizer

## Hill Climbing

Let's start with keeping just one state in memory! That's right we may solve problems with *O(1)* memory! The obvious question, however, is where to go from it! A logical, but greedy, approach would be to consider the best next possible state (highest-valued child). This mirrors climbing straight up a hill instead of considering all other possible directions (and, obviously, that is how it got its name!). It is also known as *greedy local search*, nothing surprising here! As most *greedy* algorithms, this is *not complete*! This approach can easily get stuck in local maxima (the top of *a* hill instead of the highest possible hill).

In [ ]:
def hill_climbing(problem, verbose=False):
    '''Hill climbing search implementation.'''
    current_state = problem.init_state
    current_value = problem.heuristic(current_state)
    if verbose: visualizer = Visualizer(problem)
    while True:
        if verbose: visualizer.visualize([current_state])
        next_state, next_value = None, None
        for action in problem.actions(current_state):
            new_state = problem.result(current_state, action)
            new_value = problem.heuristic(new_state)
            if next_value is None or next_value > new_value:
                next_state, next_value = new_state, new_value
        if current_value <= next_value: return current_state
        current_state, current_value = next_state, next_value

Let's try solving the 8-queens puzzle with hill climbing search!

In [ ]:
problem = EightQueens.new_random_instance()
problem.heuristic(hill_climbing(problem, verbose=True))

Frontier at step 1

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬛⬜
⬜⬜⬜⬜⬜⬜⬜⬜
--------------------------------------------------------------------------------
Frontier at step 2

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬛⬜
⬜⬜⬜⬜⬜⬜⬜⬜
--------------------------------------------------------------------------------
Frontier at step 3

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜
⬜⬜⬜⬜⬛⬜⬜⬜
--------------------------------------------------------------------------------


2

What happens if you get *stuck*? Well... **TRY AGAIN!**

### Random-Restart Hill Climbing

Simply put, for problems that can be formulated with a random initial state (like our formulation of the 8-queens puzzle), trying again will get you somewhere else. This is called *random-restart hill climbing*. It is a form of a random walk which is *trivially complete* because with enough trials, you are guaranteed to start from a goal state! With that been said, it is worth noting that random-restart hill climbing is *much* more efficient than completely random walks.

In [ ]:
def random_restart_hill_climbing(problem_gen, verbose=False):
    '''Random-restart hill climbing search implementation.'''
    while True:  # Keep repeating till the problem is solved
        problem = problem_gen()  # Generate a new problem instance (only works if each problem instance is different from the previous, e.g., random initial state)
        solution_state = hill_climbing(problem, verbose)  # Try to solve the problem instance with hill climbing
        if problem.goal_test(solution_state):  return solution_state  # If succeeded, return the solution

In [ ]:
random_restart_hill_climbing(EightQueens.new_random_instance, verbose=True)

Frontier at step 1

⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬛
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
--------------------------------------------------------------------------------
Frontier at step 2

⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬛⬜
⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
--------------------------------------------------------------------------------
Frontier at step 3

⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
--------------------------------------------------------------------------------
Frontier at step 4

⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬛⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
--------------------------------------------------------------------------------


(7, 1, 3, 8, 6, 4, 2, 5)

### Simulated Annealing

Another approach to mitigate getting stuck is to drop the *best next* requirement. There are a complete family of such algorithms called *stochastic hill climbing* algorithms discussing random strategies to choose the next state. One interesting variation, called *simulated annealing*, even allows downward actions! It is analogous to the *annealing* process in *metallurgy*, from which it gets its name. The idea is to *shake things up* a bit in the start, i.e., be more permissive of bad moves, to better explore the landscape. Then, it cools off as time goes on, i.e., bad moves get less and less tolerated.

In [ ]:
from random import choice, random
from math import exp
from itertools import count

def simulated_annealing(problem, schedule, verbose=False):
    '''Simulated annealing search implementation.'''
    current_state = problem.init_state
    current_value = problem.heuristic(current_state)
    if verbose: visualizer = Visualizer(problem)
    for t in count():
        if verbose: visualizer.visualize([current_state])
        T = schedule(t)  # A function that determines the "temperature" (acceptability of a bad state) as a function of the step count
        if current_value is 0 or T is 0: return current_state  # Return if a goal state is found or if the temperature hits 0
        next_states = [problem.result(current_state, action) for action in problem.actions(current_state)]  # Generate all possible next states
        while True:  # Repeat the following till the current state is updated
            next_state = choice(next_states)  # Choose a random next state
            next_value = problem.heuristic(next_state)
            delta = current_value - next_value
            if delta > 0 or random() < exp(delta / T):  # Accept the randomly chosen state immediately if it is better than the current state or with a probability (exponentially) proportional to the temperature and how bad it is
                current_state, current_value = next_state, next_value
                break

In [ ]:
simulated_annealing(EightQueens.new_random_instance(), lambda t: exp(-t), verbose=True)

Frontier at step 1

⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬛⬜
--------------------------------------------------------------------------------
Frontier at step 2

⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬛⬜⬜⬜⬜⬛⬜
--------------------------------------------------------------------------------
Frontier at step 3

⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬛⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬛⬜⬜⬜⬜⬜⬜
--------------------------------------------------------------------------------
Frontier at step 4

⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬛⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
--------------------------------------------------------------------------------
Frontier at step 5

⬜⬜⬛⬜⬜⬜⬛⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
--------------------------------------------------------------------------------
Frontier at step 6

⬜⬜⬜⬜⬜⬜⬛⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
-------------------------------------------

(6, 4, 7, 1, 3, 5, 2, 8)

## Requirement

Let's try local search with more than just one state in memory! One algorithm that adopts this mentality is *local beam search*. It keeps $k$ states in memory and at each step it chooses the best $k$ of all the successors of the current $k$ states.

You are required to write Python code that implements the *local beam search* algorithm and apply it to the 8-queens puzzle problem and compare it with *hill climbing*!

**Estimated time for this exercise is 45 minutes!**

In [ ]:
def Local_Beam_Search(k, problem, verbose=False):
    '''Local Beam  search implementation.'''
    current_state = problem.init_state
    scanned_States = [(current_state)]

    if verbose: visualizer = Visualizer(problem)
    while True:
        if verbose: visualizer.visualize(scanned_States)
        for state in scanned_States:
          if problem.heuristic(state) == 0:
            return state
          else:
            available_States = []
            for state in scanned_States:
              avail = [problem.result(state, action) for action in problem.actions(state)]
              available_States.extend(avail)
            temp = available_States[0:k]
            for i in available_States:
              for j in range(k):
                if problem.heuristic(i) < problem.heuristic(temp[j]):
                  temp2 = i
                  i = temp[j]
                  temp[j] = temp2
                  j = k-1
            scanned_States = temp

In [ ]:
Local_Beam_Search(4, EightQueens.new_random_instance(),verbose=True)

Frontier at step 1

⬜⬜⬜⬜⬛⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜
--------------------------------------------------------------------------------
Frontier at step 2

⬜⬜⬜⬜⬛⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜

⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬛⬜⬜⬜
⬛⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜

⬜⬜⬜⬜⬛⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜

⬜⬜⬜⬜⬛⬛⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬛⬜
--------------------------------------------------------------------------------
Frontier at step 3

⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬛⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜

⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬜⬜⬛⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜

⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬛
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜

⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬛
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜
--------------------------------------------------------------------------------


(4, 7, 5, 3, 1, 6, 8, 2)

In [ ]:
Local_Beam_Search(7, EightQueens.new_random_instance(),verbose=True)

Frontier at step 1

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬛
--------------------------------------------------------------------------------
Frontier at step 2

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬛

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬛

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬛

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬛

⬜⬜⬛⬜⬛⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬛

⬜⬜⬛⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬛

⬜⬜⬜⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬛⬛⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬛
--------------------------------------------------------------------------------
Frontier at step 3

⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜

⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜

⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜
⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬛⬜⬜⬜

(1, 6, 8, 3, 7, 4, 2, 5)

## _Optional_ Pythonic Implementations

Following are compact, so-called _pythonic_, re-implementations of some of the codes above. These are completely optional and intended for readers interested in advanced Python shortcuts and optimizations.

**DO NOT WASTE TIME UNDERSTANDING IT TILL YOU ARE DONE WITH THE REQUIREMENT!**

In [ ]:
from random import randint

class EightQueens(Problem):
    '''8-Queens Puzzle problem formulation.'''

    def __init__(self):
        self.init_state = tuple(randint(1, 8) for _ in range(8))

    def actions(self, state):
        return ((i, new_placement) for i, placement in enumerate(state) for new_placement in range(1, 9) if new_placement is not placement)

    def result(self, state, action):
        new_state = list(state)
        new_state[action[0]] = action[1]
        return tuple(new_state)

    def goal_test(self, state):
        return self.heuristic(state) is 0

    def step_cost(self, state, action):
        return 1

    def heuristic(self, state):
        return sum((placement1 is placement2) + (abs(int(placement1) - int(placement2)) is j+1) \
                   for i, placement1 in enumerate(state) for j, placement2 in enumerate(state[i + 1:]))

def _eight_queens_visualizer(problem, state):
    '''Custom visualizer for the eight queens puzzle problem.'''
    for i in range(1, 9):
        for j in range(8):
            print('⬛' if state[j] is i else '⬜', end='')
        print()

_visualizers[EightQueens] = _eight_queens_visualizer

In [ ]:
def hill_climbing(problem, verbose=False):
    '''Hill climbing search implementation.'''
    current_state = problem.init_state
    current_value = problem.heuristic(current_state)
    if verbose: visualizer = Visualizer(problem)
    while True:
        if verbose: visualizer.visualize([current_state])
        next_state, next_value = min(map(lambda state: (state, problem.heuristic(state)),
                                         (problem.result(current_state, action) for action in problem.actions(current_state))),
                                     key=lambda state_value_pair: state_value_pair[1])
        if current_value <= next_value: return current_state
        current_state, current_value = next_state, next_value

## _Optional_ $n$-Queens Puzzle

For the sake of completeness, this is the same generalized formulation for the $n$-queens puzzle problem that enables solving $n$-queens puzzles of any size. This is completely optional.

**DO NOT WASTE TIME UNDERSTANDING IT TILL YOU ARE DONE WITH THE REQUIREMENT!**

In [ ]:
from random import randint

class NQueens(Problem):
    '''8-Queens Puzzle problem formulation.'''

    def __init__(self, init_state):
        self.n = len(init_state)
        self.init_state = init_state

    def actions(self, state):
        return ((i, new_placement) for i, placement in enumerate(state) for new_placement in range(1, len(state) + 1) if new_placement is not placement)

    def result(self, state, action):
        new_state = list(state)
        new_state[action[0]] = action[1]
        return tuple(new_state)

    def goal_test(self, state):
        return self.heuristic(state) is 0

    def step_cost(self, state, action):
        return 1

    def heuristic(self, state):
        return sum((placement1 is placement2) + (abs(int(placement1) - int(placement2)) is j+1) \
                   for i, placement1 in enumerate(state) for j, placement2 in enumerate(state[i + 1:]))

    @classmethod
    def new_random_instance(cls, n):
        return cls(tuple(randint(1, n) for _ in range(n)))


def _n_queens_visualizer(problem, state):
    '''Custom visualizer for the eight queens puzzle problem.'''
    n = len(state)
    for i in range(1, n + 1):
        for j in range(n):
            print('⬛' if state[j] is i else '⬜', end='')
        print()

_visualizers[NQueens] = _n_queens_visualizer

In [ ]:
random_restart_hill_climbing(lambda: NQueens.new_random_instance(9), verbose=True)

Frontier at step 1

⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬛⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬛⬜⬜⬜⬜⬛⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜⬜
--------------------------------------------------------------------------------
Frontier at step 2

⬛⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬛⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜⬛
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜⬜
--------------------------------------------------------------------------------
Frontier at step 3

⬛⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬛⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜⬛
⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜⬜
--------------------------------------------------------------------------------
Frontier at step 4

⬛⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬛⬜
⬜⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜⬜
--------------------------------------------------------------------------------
Frontier at step 5

⬛⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬛
⬜⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬛⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜⬜
--------------------------------------------------------------------------------


(1, 6, 4, 2, 8, 3, 9, 7, 5)